In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import pandas as pd
from pandas import DataFrame,Series
import matplotlib.pyplot as plt
import numpy as np

# Pandas私房手册-选取和赋值

## 选取

### 交换列的位置

这里只对一些需要注意，容易出错的地方进行说明。先创建一个`dataframe`：

In [9]:
dates = pd.date_range('1/1/2000', periods=8)
df = pd.DataFrame(
    np.random.randn(8, 4), index=dates, columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
2000-01-01,1.272911,-0.350037,0.699342,-0.568944
2000-01-02,-1.584838,0.598701,-0.458136,-0.603379
2000-01-03,-0.589415,-1.128391,1.086598,-0.953534
2000-01-04,0.139157,-1.803964,0.216849,-1.339922
2000-01-05,0.545026,-0.675006,-1.933742,-0.578932
2000-01-06,1.646492,-0.396650,0.292728,1.176558
2000-01-07,-0.468386,0.986369,-2.023729,-1.491446
2000-01-08,1.134426,-0.375839,-0.916211,-1.238013


交换两列的值，可以直接像下面这样写（也可以交换多列的位置）：

In [10]:
df[['B', 'A']] = df[['A', 'B']]
df

,A,B,C,D
2000-01-01,-0.350037,1.272911,0.699342,-0.568944
2000-01-02,0.598701,-1.584838,-0.458136,-0.603379
2000-01-03,-1.128391,-0.589415,1.086598,-0.953534
2000-01-04,-1.803964,0.139157,0.216849,-1.339922
2000-01-05,-0.675006,0.545026,-1.933742,-0.578932
2000-01-06,-0.396650,1.646492,0.292728,1.176558
2000-01-07,0.986369,-0.468386,-2.023729,-1.491446
2000-01-08,-0.375839,1.134426,-0.916211,-1.238013


要注意：当使用`.loc`和`.iloc`选取数据时，`panda`对齐所有轴。因此，下面这样的写法不会起作用，因为列对齐在值赋值之前,即会先根据列索引将A,B两列对齐，然后再赋值，相当于A，B根本没有交换位置：

In [12]:
df.loc[:, ['B', 'A']] = df[['A', 'B']]
df

,A,B,C,D
2000-01-01,-0.350037,1.272911,0.699342,-0.568944
2000-01-02,0.598701,-1.584838,-0.458136,-0.603379
2000-01-03,-1.128391,-0.589415,1.086598,-0.953534
2000-01-04,-1.803964,0.139157,0.216849,-1.339922
2000-01-05,-0.675006,0.545026,-1.933742,-0.578932
2000-01-06,-0.396650,1.646492,0.292728,1.176558
2000-01-07,0.986369,-0.468386,-2.023729,-1.491446
2000-01-08,-0.375839,1.134426,-0.916211,-1.238013


正确的做法是，先转换成`numpy`数组：

In [15]:
df.loc[:, ['B', 'A']] = df[['A', 'B']].to_numpy()  # 或者df[['A', 'B']].values
df

,A,B,C,D
2000-01-01,1.272911,-0.350037,0.699342,-0.568944
2000-01-02,-1.584838,0.598701,-0.458136,-0.603379
2000-01-03,-0.589415,-1.128391,1.086598,-0.953534
2000-01-04,0.139157,-1.803964,0.216849,-1.339922
2000-01-05,0.545026,-0.675006,-1.933742,-0.578932
2000-01-06,1.646492,-0.396650,0.292728,1.176558
2000-01-07,-0.468386,0.986369,-2.023729,-1.491446
2000-01-08,1.134426,-0.375839,-0.916211,-1.238013


### 使用`loc`和`iloc`选取的注意事项

`loc`通过整型索引选取数据时，如果索引是有序的，则切片的起始结束标签可以在索引的范围之外或者是索引的缺失值，如果不是有序的，则会报错：

In [8]:
s = pd.Series(list('abcde'), index=[0, 3, 2, 5, 4])

try:
    s.loc[:6]
except Exception as e:
    print(f"{e.__class__.__name__}: {e}")

s.sort_index()[:6]

KeyError: 6


0    a
2    c
3    b
4    e
5    d
dtype: object

对于`iloc`，处理的方式和原生的`python`相似，如果切片的起始结束标签之间不包含任何数据，则会返回一个为空的结果，但是如果不是切片，而是一个单独的标签或者标签列表不在边界之内，则会抛出一个`IndexError`的告警。

In [10]:
s

s.iloc[6:10] # 索引不存在，返回空

try:
    s.iloc[6]
except Exception as e:
    print(f"{e.__class__.__name__}: {e}") # 超过界限，抛出错误

try:
    s.iloc[[6, 7, 8]]
except Exception as e:
    print(f"{e.__class__.__name__}: {e}") # 超过界限，抛出错误

0    a
3    b
2    c
5    d
4    e
dtype: object

Series([], dtype: object)

IndexError: single positional indexer is out-of-bounds
IndexError: positional indexers are out-of-bounds


### 通过`get()`方法来选取

`Series`和`DataFrame`对象都有一个`get`方法可以设置默认值，如果没有设置，则返回空值：

In [28]:
s = pd.Series([1, 2, 3], index=['a', 'b', 'c'])
s.get('x', -1)

-1

注意，`dataframe`的行为，如果某列不存在，返回的默认值是标量：

In [36]:
df
df.get('A')
df.get('Z', 0)

,A,B,C,D
2000-01-01,1.000000,-0.501623,-0.184873,0.580874
2000-01-02,-0.718034,-0.384270,-1.979562,0.805093
2000-01-03,-1.523666,-0.867515,-0.362976,-0.639871
2000-01-04,-1.157796,-0.736984,1.661237,0.483189
2000-01-05,0.278545,-1.291611,-0.584248,-1.553789
2000-01-06,0.478416,-0.986069,-0.722679,0.620085
2000-01-07,0.071753,0.265143,0.555044,0.303586
2000-01-08,-0.567021,0.387110,-1.946723,-1.357457


2000-01-01    1.000000
2000-01-02   -0.718034
2000-01-03   -1.523666
2000-01-04   -1.157796
2000-01-05    0.278545
2000-01-06    0.478416
2000-01-07    0.071753
2000-01-08   -0.567021
Freq: D, Name: A, dtype: float64

0

### 通过`take()`方法选取

`Index`、`Series`和`DataFrame`均提供了`take()`方法，根据传入的位置选取数据，可以传入数组，或者`slice`对象，它还接受负整数，表示从后向前选取,注意不能传入标量：

In [49]:
df

,A,B,C,D
2000-01-01,1.000000,-0.501623,-0.184873,0.580874
2000-01-02,-0.718034,-0.384270,-1.979562,0.805093
2000-01-03,-1.523666,-0.867515,-0.362976,-0.639871
2000-01-04,-1.157796,-0.736984,1.661237,0.483189
2000-01-05,0.278545,-1.291611,-0.584248,-1.553789
2000-01-06,0.478416,-0.986069,-0.722679,0.620085
2000-01-07,0.071753,0.265143,0.555044,0.303586
2000-01-08,-0.567021,0.387110,-1.946723,-1.357457


In [52]:
df.take([0, 1])
df.take([0, -1])
df.take(slice(0, 8, 2))

,A,B,C,D
2000-01-01,1.000000,-0.501623,-0.184873,0.580874
2000-01-02,-0.718034,-0.384270,-1.979562,0.805093


,A,B,C,D
2000-01-01,1.000000,-0.501623,-0.184873,0.580874
2000-01-08,-0.567021,0.387110,-1.946723,-1.357457


,A,B,C,D
2000-01-01,1.000000,-0.501623,-0.184873,0.580874
2000-01-03,-1.523666,-0.867515,-0.362976,-0.639871
2000-01-05,0.278545,-1.291611,-0.584248,-1.553789
2000-01-07,0.071753,0.265143,0.555044,0.303586


还有两点要注意：
- 不要传递布尔值给`take()`方法，因为Python中布尔值是数值，返回的结果不是你期望的。
- `take()`方法性能稍高于`iloc`方法。

In [24]:
df = pd.DataFrame(np.random.rand(10000, 4), columns = ['A', 'B', 'C', 'D'])
idx = np.random.randint(0, 1000, 50)

In [25]:
%timeit df.take(idx)

394 µs ± 25.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [26]:
%timeit df.iloc[idx]

421 µs ± 11.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### 通过`lookup()`方法选取

有时，希望提取给定的行标签和列标签序列的一组值，此时可以使用`lookup()`方法，它会返回一个NumPy数组。注意和`loc`的区别，`lookup`选取行列标签对应的单个的值，相当于`loc`方法结果的对角线，例如：

In [69]:
dflookup = pd.DataFrame(np.random.rand(20, 4), columns = ['A', 'B', 'C', 'D'])
dflookup.lookup(list(range(0, 10, 2)), ['B', 'C', 'A', 'B', 'D'])
dflookup.loc[[0, 2, 4, 6, 8], ['B', 'C', 'A', 'B', 'D']]

array([0.03700312, 0.82676519, 0.74746458, 0.28855079, 0.59330729])

,B,C,A,B,D
0,0.037003,0.168861,0.346790,0.037003,0.649481
2,0.457487,0.826765,0.757205,0.457487,0.626653
4,0.052790,0.932258,0.747465,0.052790,0.638031
6,0.288551,0.050584,0.874867,0.288551,0.755818
8,0.756434,0.590833,0.630805,0.756434,0.593307


### 使用可调用对象进行选取

`.loc`、`.iloc`以及`[]`可以接受可调用对象进行选取。可调用对象必须具有一个参数(`Series`或`DataFrame`)，这种方式最大的作用是用于链式编程而不需要中间变量：

In [63]:
df1 = pd.DataFrame(
    np.random.randn(6, 4), index=list('abcdef'), columns=list('ABCD'))
df1
# 下面的语句表示同事满足A和B列都大于0
df1[df1.B > 0].loc[lambda df: df.A > 0, :]

,A,B,C,D
a,-1.106948,-1.792965,1.889929,-0.033923
b,-1.417454,0.279046,0.024982,-0.954854
c,0.045547,-2.570786,0.469330,-1.016766
d,0.288235,-1.350184,1.861442,-1.570477
e,0.611115,0.505929,1.233052,-3.011305
f,-0.175252,-0.602892,1.582511,1.001608


,A,B,C,D
e,0.611115,0.505929,1.233052,-3.011305


### 根据数据类型进行选取

可以利用`select_dtypes`方法针对数据类型选取列，可以先根据`dtypes`属性查看每一列的数据类型。有`include`和`exclude`两个参数，分别表示包含的数据类型和排除的数据类型，可以传入列表，包含多种数据类型：

In [58]:
df = DataFrame([[1, 'b', 'c'], [2, 2, 'a'], [3, 4, 'd']])
df
df.select_dtypes(exclude=object)

,0,1,2
0,1,b,c
1,2,2,a
2,3,4,d


,0
0,1
1,2
2,3


### `reindex`也是一种选取

个人认为把`reindex`看成选取数据的一种方式更有利于理解和使用`reindex`方法，而不是简单的认为替换整个原索引，如果新索引的标签在原索引中存在，则取该标签的值，如果不存在，则设置为`NaN`：

In [60]:
s = Series([1, 2, 3])
s.reindex([1, 2, 3])

1    2.0
2    3.0
3    NaN
dtype: float64

要注意的是，如果原索引里面有重复的标签，调用`reindex`方法会抛出`ValueError`错误，按上面的方法就容易理解（因为新索引的标签在原索引中重复，所以不知道选哪一个，所以报错）：

In [68]:
s = pd.Series(np.arange(4), index=['a', 'a', 'b', 'c'])
try:
    s.reindex(['c', 'd'])
except Exception as e:
    print(f"{e.__class__.__name__}: {e}")

ValueError: cannot reindex from a duplicate axis


此时可以使用`Index`索引的`intersection`方法先求取交集，再`reindex`。注意，如果交集里面仍然有重复的标签，依然会报错：

In [69]:
labels = ['c', 'd']
s.loc[s.index.intersection(labels)].reindex(labels)

c    3.0
d    NaN
dtype: float64

### 选取随机的样本

可以通过`.sample()`方法选取随机的样本，其中`n`参数代表选取几个样本，`frac`参数代表按照百分比选取样本：

In [71]:
s = Series([0, 1, 2, 3, 4, 5])
s.sample(n=3)
s.sample(frac=0.5)

1    1
3    3
2    2
dtype: int64

3    3
0    0
4    4
dtype: int64

默认是无放回的选取，即不能重复选取同一个值，但是可以通过将`replace`设置为`True`，此时是有放回的选取，即可以重复选取同一个值：

In [22]:
s.sample(n=6, replace=False)
s.sample(n=6, replace=True)

5    5
2    2
4    4
3    3
0    0
1    1
dtype: int64

2    2
2    2
4    4
4    4
0    0
0    0
dtype: int64

默认情况下，每一行被选中的概率是相等的，如果希望各行具有不同的概率，可以通过`weights`参数设置权重，可以是列表、NumPy数组或序列，但必须与要采样的对象的长度相同。缺失值将被视为权重为零，不允许使用`inf`值。如果权重之和不等于1，则通过将所有权重除以权重之和来重新标准化。例如：

In [23]:
s = pd.Series([0, 1, 2, 3, 4, 5])
example_weights = [0, 0, 0.2, 0.2, 0.2, 0.4]
s.sample(n=3, weights=example_weights)
# 虽然第一个的权重是0.5，但是其它都是0，权重之和是0.5，所以第一个的实际权重为0.5/0.5=1
example_weights2 = [0.5, 0, 0, 0, 0, 0]
s.sample(n=1, weights=example_weights2)

4    4
2    2
5    5
dtype: int64

0    0
dtype: int64

如果是`dataframe`的话，可以将某列作为权重，只需要将该列的标签传入`weights`参数：

In [24]:
df2 = pd.DataFrame({'col1': [9, 8, 7, 6], 'weight_column': [0.5, 0.4, 0.1, 0]})
df2.sample(n=3, weights='weight_column')

,col1,weight_column
1,8,0.4
0,9,0.5
2,7,0.1


最后，`sample`还可以通过`axis`和`random_state`来随机选取列以及设定随机种子。

### 布尔索引

布尔索引需要注意2点：
1. 不能用`and`，`or`，`not`，只能使用对应的`&`，`|`，`~`符号。
2. 比较表达式要加括号，比如`a > 0 & b > 0`，不加括号的话，`pandas`会这样计算`a > (0 & b) >0`。
3. 直接在切片中使用布尔索引时（即不适用`loc`,`iloc`等方法），选取的是行而不是列。

#### 通过列表推导和`map()`方法创建布尔索引

列表推导和`map`映射可以适应更加复杂的选取条件：

In [16]:
df2 = pd.DataFrame({
    'a': ['one', 'one', 'two', 'three', 'two', 'one', 'six'],
    'b': ['x', 'y', 'y', 'x', 'y', 'x', 'x'],
    'c': np.random.randn(7)
})
df2
criterion = df2['a'].map(lambda x:x.startswith('t'))
df2[criterion]
# 或者使用列表推导，效果是一样的
# df2[[x.startswith('t') for x in df2['a']]]

,a,b,c
0,one,x,-0.102695
1,one,y,-0.915172
2,two,y,-0.854199
3,three,x,2.176532
4,two,y,0.671609
5,one,x,-0.800101
6,six,x,0.420763


,a,b,c
2,two,y,-0.854199
3,three,x,2.176532
4,two,y,0.671609


#### 通过`isin()`方法创建布尔索引

可以通过`isin()`方法创建一个布尔索引来进行选取，`Series`，`DataFrame`，`Index`对象都有`isin()`方法：

In [40]:
s = pd.Series(np.arange(5), index=np.arange(5)[::-1], dtype='int64')
s[s.index.isin([2, 4, 6])]
# 注意和reindex的区别
s.reindex([2, 4, 6])

4    0
2    2
dtype: int64

2    2.0
4    0.0
6    NaN
dtype: float64

`MutiIndex`层级索引的`isin()`要复杂一点，可以通过元组代表不同层级的标签，也可以通过`level`参数指定某一层：

In [43]:
s_mi = pd.Series(
    np.arange(6), index=pd.MultiIndex.from_product([[0, 1], ['a', 'b', 'c']]))
s_mi
s_mi.iloc[s_mi.index.isin([(1, 'a'), (2, 'b'), (0, 'c')])]
s_mi.iloc[s_mi.index.isin(['a', 'b', 'd', 'e'], level=1)]

0  a    0
   b    1
   c    2
1  a    3
   b    4
   c    5
dtype: int32

0  c    2
1  a    3
dtype: int32

0  a    0
   b    1
1  a    3
   b    4
dtype: int32

如果原始对象是`dataframe`，则可以传递一个字典，来指定相应的列是否包含哪些数据，没有指定的列统一为`False`：

In [112]:
df = pd.DataFrame({
    'vals': [1, 2, 3, 4],
    'ids': ['a', 'b', 'f', 'n'],
    'ids2': ['a', 'n', 'c', 'n']
})
df
df.isin({'vals': [1, 3], 'ids': ['a', 'b']})

,vals,ids,ids2
0,1,a,a
1,2,b,n
2,3,f,c
3,4,n,n


,vals,ids,ids2
0,True,True,False
1,False,True,False
2,True,False,False
3,False,False,False


结合`all`和`any`方法，可以快速的根据需要选择子集：

In [111]:
mask = df.isin({'vals': [1, 3], 'ids': ['a', 'b'], 'ids2': ['a', 'c']}).all(1)  # 1代表1轴，沿着水平方向
mask
df[mask]  # 注意，当直接使用布尔值，过滤的是行不是列

0     True
1    False
2    False
3    False
dtype: bool

,vals,ids,ids2
0,1,a,a


### 过滤重复数据

`Series`，`DataFrame`和`Index`对象都有`duplicated`和`drop_duplicates`两个方法，需要注意的是，可以传入列表来判断重复项：

In [79]:
df2 = pd.DataFrame({
    'a': ['one', 'one', 'two', 'two', 'two', 'three', 'four'],
    'b': ['x', 'y', 'x', 'y', 'x', 'x', 'x'],
    'c': np.random.randn(7)
})
df2
df2.duplicated(['a', 'b'])
df2.drop_duplicates('b')

,a,b,c
0,one,x,0.057809
1,one,y,-0.164728
2,two,x,1.918963
3,two,y,0.791057
4,two,x,0.017655
5,three,x,0.656256
6,four,x,-1.341303


0    False
1    False
2    False
3    False
4     True
5    False
6    False
dtype: bool

,a,b,c
0,one,x,0.057809
1,one,y,-0.164728


### get_indexer

暂时还没有碰到这个方法使用的场景，它实际上是返回一列值在给定索引中的位置，没有找到则返回-1：

In [6]:
s = pd.Series(['c', 'a', 'b', 'b', 'c', 'a', 'd'])
idx = pd.Index(['a', 'c'])
idx.get_indexer(s)

array([ 1,  0, -1, -1,  1,  0, -1], dtype=int64)

## 赋值

### 使用`assign`方法对列进行赋值

除了`[]`,`iloc`和`loc`，还可以通过`assign`方法对列进行赋值，需要注意的是，`[]`,`iloc`和`loc`直接修改原对象，`assign`返回一个副本，`assign`最大的好处是不需要中间变量，因此可以方便的进行链式编程：

In [9]:
pd.read_excel('excel_demo.xlsx').assign(new=[1, 2])

,Unnamed: 0,col1,col2,new
0,row1,1,2,1
1,row2,3,4,2


`assign`方法可以通过函数来创建列，此时传入函数的是`dataframe`对象本身：

In [11]:
pd.read_excel('excel_demo.xlsx').assign(new1=[1, 2]).assign(
    new2=lambda df: df.new1 + 1)

,Unnamed: 0,col1,col2,new1,new2
0,row1,1,2,1,2
1,row2,3,4,2,3


从Python 3.6开始，保留了`**kwargs`的顺序。因此0.23版本以后，`assign()`中后创建的列可以引用较早创建的列，就不用再像上面那样反复调用`assign`，而是可以一次搞定：

In [14]:
pd.read_excel('excel_demo.xlsx').assign(new1=[1, 2],
                                        new2=lambda df: df.new1 + 1)

,Unnamed: 0,col1,col2,new1,new2
0,row1,1,2,1,2
1,row2,3,4,2,3


### 通过属性对列赋值

也可以通过属性赋值，但是如果通过属性赋值的话，注意不能创建新列，0.21版本以后会有`UserWarning`警告。但是可以通过属性覆盖已有的列，如下：

In [81]:
# 出现警告，没有创建新的C列
df.C = list(range(len(df.index)))
df
# 原有的A列被覆盖
df.A = list(range(len(df.index)))
df

,0,1,2
0,1,b,c
1,2,2,a
2,3,4,d


,0,1,2
0,1,b,c
1,2,2,a
2,3,4,d


### `loc`和`iloc`结合字典对行进行赋值

`loc`和`iloc`除了使用序列给行赋值，还可以使用字典，注意，`value`的类型需要和列的类型一致，否则不会使用字典的`value`，而是直接用`key`来赋值，如下：

In [115]:
df = DataFrame(np.random.rand(3, 2), columns=['A', 'B'])
df
df.iloc[0] = {'A': 'c', 'B': 'd'}
df
df.iloc[0] = {'A': 42, 'B': 24}
df

,A,B
0,0.095430,0.078869
1,0.712634,0.554170
2,0.100994,0.855640


,A,B
0,A,B
1,0.712634,0.55417
2,0.100994,0.85564


,A,B
0,42,24
1,0.712634,0.55417
2,0.100994,0.85564


但是有个很有趣的现象，如果使用字典给列赋值（不管是`loc`还是`iloc`，只要是`[:, 标签或者位置]`的形式），此时只是简单的赋给字典的`key`，而不是`value`：

In [116]:
df
df.iloc[:, 0] = {'a':11, 'b':22, 'c':33}
df

,A,B
0,42,24
1,0.712634,0.55417
2,0.100994,0.85564


,A,B
0,a,24
1,b,0.55417
2,c,0.85564


### 通过`where()`方法赋值

可以通过`where()`方法创建一个`mask`蒙罩，蒙罩保留原始数据对象的形状，你可以把它想象成一块布，为`True`的地方挖了个洞露出来，为`False`的则会被设置为`NaN`，`Dataframe`对象的布尔索引结果和蒙罩是一样的，但`Series`的布尔索引不会保留`False`位置的值，另外，还有一个`mask()`的方法，它和`where()`正好相反：

In [3]:
df = DataFrame(np.random.rand(3, 2), columns=['A', 'B'])
df
df[df > 0.5]
df.where(df > 0.5)
df.mask(df > 0.5)

,A,B
0,0.098276,0.118748
1,0.393004,0.124850
2,0.912728,0.280121


,A,B
0,NaN,NaN
1,NaN,NaN
2,0.912728,NaN


,A,B
0,NaN,NaN
1,NaN,NaN
2,0.912728,NaN


,A,B
0,0.098276,0.118748
1,0.393004,0.124850
2,NaN,0.280121


In [110]:
s[s > 0]
s.where(s > 0)
s.mask(s > 0)

a    1
b    2
c    3
dtype: int64

a    1
b    2
c    3
dtype: int64

a   NaN
b   NaN
c   NaN
dtype: float64

`where()`还有一个`other`参数，可以设定那些为`False`的值，注意：`other`只能为`Series`，`DataFrame`和可调用对象，传入列表的话会陷入一个无限循环的报错信息当中：

In [9]:
df = DataFrame(np.random.randn(4, 3))
df
# 所有为False，即小于0的数取反变为正数
df.where(df > 0, -df)

,0,1,2
0,-0.559953,0.178092,1.245123
1,0.244395,-0.192335,0.153819
2,0.885130,0.819614,0.727890
3,0.369024,-0.786440,0.077868


,0,1,2
0,0.559953,0.178092,1.245123
1,0.244395,0.192335,0.153819
2,0.885130,0.819614,0.727890
3,0.369024,0.786440,0.077868


还可以通过设置`axis`参数来进行针对行或者列的替换，一般都是配合`other`参数使用，不过这里要注意的是，此时`other`参数只能是`Series`对象，如果使用的是列表，会陷入一个无限循环的报错，个人觉得这应该是一个小bug(新版本已经修复)。比如，你想将每一列小于0的数用'a'代替，第二行用'b',第三行用'c'。那么可以这样做：

In [48]:
df.columns = ['a', 'b', 'c']
df
s = Series(['a', 'b', 'c'], index=df.columns)
df.where(df > 0, s, axis='columns')

,a,b,c
0,-0.559953,0.178092,1.245123
1,0.244395,-0.192335,0.153819
2,0.885130,0.819614,0.727890
3,0.369024,-0.786440,0.077868


,a,b,c
0,a,0.178092,1.245123
1,0.244395,b,0.153819
2,0.88513,0.819614,0.72789
3,0.369024,b,0.077868


In [11]:
%timeit df.where(df > 0, s, axis='columns')

839 µs ± 17.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


这相当于以下的代码，但是要比使用`apply`稍微快：

In [12]:
%timeit df.apply(lambda x, y: x.where(x > 0, y), y=s, axis='columns')

2.35 ms ± 64.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


要注意的是，此时的`Series`对象的标签要和行或者列的标签一致，否则不会进行替换，仍然会是`NaN`:

In [13]:
s1 = Series([1, 2, 3])
df.where(df > 0, s1, axis='columns')

,a,b,c
0,NaN,0.178092,1.245123
1,0.244395,NaN,0.153819
2,0.885130,0.819614,0.727890
3,0.369024,NaN,0.077868


`cond`和`other`参数还可以是一个可调用对象（不过`cond`参数是可调用对象好像没有什么意义），此时传入该函数（说可调用对象太拗口）的参数是调用`where`方法的原始`dataframe`对象，组合起来可以满足复杂的需求，比如，上面的`dataframe`对象，你想将每一列小于0的数用该列的最大值来代替：

In [36]:
df

,a,b,c
0,-0.559953,0.178092,1.245123
1,0.244395,-0.192335,0.153819
2,0.885130,0.819614,0.727890
3,0.369024,-0.786440,0.077868


In [37]:
# 注意，df.max()返回的是每一列的最大值组成的一个Series，必须要设置axis=1，默认是None
df.where(df > 0, lambda df: df.max(), axis=1)

,a,b,c
0,0.885130,0.178092,1.245123
1,0.244395,0.819614,0.153819
2,0.885130,0.819614,0.727890
3,0.369024,0.819614,0.077868


### 为什么赋值时经常会出现`SettingWithCopy`告警？

我们在对切片的结果进行赋值的时候，偶尔会出现`SettingWithCopy`告警，这是为什么呢？  
一般而言，切片操作返回的结果有两种，一种是视图，即是原对象的一个快照，在视图上修改就相当于修改了原对象，一种是副本，相当于一个新的对象，修改它对原对象没有影响，而操作的顺序和类型都有可能决定最后返回的是视图还是副本，通常情况下，我们对切片结果进行赋值，是希望赋值给视图而不是副本。当我们有可能在对副本进行赋值的时候，就会出现这个告警。那么怎样避免呢？  
在深入了解之前，先了解一个链式索引的概念，看下面的例子：

In [121]:
dfmi = pd.DataFrame(
    [list('abcd'), list('efgh'),
     list('ijkl'), list('mnop')],
    columns=pd.MultiIndex.from_product([['one', 'two'], ['first', 'second']]))
dfmi

one          two       
  first second first second
0     a      b     c      d
1     e      f     g      h
2     i      j     k      l
3     m      n     o      p

现在要获取第二列，下面两种方法中第一种就属于链式方法：

In [122]:
dfmi['one']['second']
dfmi.loc[:, ('one', 'second')]

0    b
1    f
2    j
3    n
Name: second, dtype: object

0    b
1    f
2    j
3    n
Name: (one, second), dtype: object

两种方法返回同样的结果，但是第二种更好更高效。因为第一种方法，`dfmi['one']`选择列的第一层并返回一个单索引的`dataframe`。然后，继续对结果进行选择`dfmi_with_one['second']`，选择索引为'second'的列，panda将这些操作视为单独的事件，在底层，其实是调用了2次`__getitem__`方法。  
而第二种，`df.loc[:，('one'，'second')]`是将一个嵌套的元组`(slice(None)，('one'，'second'))`一次性传递给`__getitem__`，显然`__getitem__`方法只调用了一次，因此，第二种要更高效。  
使用第一种方法进行赋值的时候，就会出现`SettingWithCopy`的警告，接下来深入了解为什么会出现这种告警：  
先看第二种方法，调用了`__setitem__`一次：
```python
dfmi.loc[:, ('one', 'second')] = value
# 底层相当于调用__setitem__，且只调用一次
dfmi.loc.__setitem__((slice(None), ('one', 'second')), value)
```
再看链式方法：
```python
dfmi['one']['second'] = value
# 先调用__getitem__，然后再调用__setitem__
dfmi.__getitem__('one').__setitem__('second', value)
```
看到没，链式方法要先调用`__getitem__`方法，再对结果调用`__setitem__`方法，那么问题来了，`pandas`是很难判断第一次`__getitem__`方法返回的到底是视图还是副本的（严格来说，应该是可以判断的，但是`pandas`不知道到底返回的结果类型-视图或者副本-是否就是用户所希望的），因此它会抛出警告来警告你，并返回一个副本。  
当然，你可以通过修改`pandas`的`mode.chained_assignment`选项来修改默认的行为，执行语句为：`pd.set_option('mode.chained_assignment','warn')`：
- `'warn'`，默认值，抛出警告
- `'raise'`，抛出异常
- `None` 不做任何处理


In [1]:
import csv

In [51]:
with open('csv_demo.csv', encoding='utf8') as f:
    reader = csv.reader(f, doublequote=False, escapechar='\\')
    for line in reader:
        print(line)

['a', 'b', 'c"h']
